In [3]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from tqdm import tqdm

from datetime import datetime


# URL Creation
Here we will create the url with which we will execute our requests to Weather API. Here we will create the url with which we will execute our requests to Weather API. We will save the response from the API in a "response" variable to later analyze it and obtain from it the data we need

In [4]:
query = 'Argentina'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'

In [5]:
response = requests.get(url_clima).json()

In [ ]:
response

In [ ]:
response.keys()

In [ ]:
response['forecast']['forecastday'][0].keys()

In [ ]:
len(response['forecast']['forecastday'][0]['hour'])

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[1].split(':')[0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

# Dataframe
Here we will create a dataframe from a function that will return the data that interests us from the API response. Each iteration of the loop will save the data in an empty list until the 24 hours of the day are complete and a list of 24 tuples is obtained. With these new data we will create a pandas dataframe from which we will obtain some fields of interest. Then we will place those fields in the message that we want Twilio to send us.

In [17]:
def get_forecast(response,i):
    
    date = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    time = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condition = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temp_c = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    will_it_rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    chance_of_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return date,time,condition,temp_c,will_it_rain, chance_of_rain

In [ ]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))

In [ ]:
datos

In [ ]:
col = ['Date','Time','Condition','Temperature','Rain','rain_chance']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Time',ascending = True)
df

In [21]:
today = df['Date'][0]
temp_max = df['Temperature'].max()
temp_min = df['Temperature'].min()
rain_hours = df[(df['Rain']==1)]
rain_hours = ', '.join(str(x) for x in rain_hours['Time'])

if df['Condition'].value_counts().idxmax() == 'Clear':
    day_forecast = df['Condition'].value_counts().idxmax() + ' ' + str(chr(0x1F308))
elif df['Condition'].value_counts().idxmax() == 'Partly cloudy':
    day_forecast = df['Condition'].value_counts().idxmax() + ' ' + str(chr(0x26C5))
elif df['Condition'].value_counts().idxmax() == 'Sunny':
    day_forecast = df['Condition'].value_counts().idxmax() + ' ' + str(chr(0x2600))
elif df['Condition'].value_counts().idxmax() == 'Patchy rain possible':
    day_forecast = df['Condition'].value_counts().idxmax() + ' ' + str(chr(0x1F326))

In [ ]:
summary = f"Summary of weather for Buenos Aires, {query}:\n" \
        f"Date: {today} {chr(0x1F4C5)}\n" \
        f"The maximum temperature will be: {temp_max} °C {chr(0x1F4C8)}\n" \
        f"The minimum temperature will be: {temp_min} °C {chr(0x1F4C9)}\n" \
        f"Today's forecast is: {day_forecast}\n" \
        f"The hours at which rain is forecast will be: {rain_hours if rain_hours else 'None'} {chr(0x2614)}\n" \

print(summary)

In [ ]:
PHONE_NUMBER

# SMS message from Twilio
What we do here is connect to our client account on Twilio and then define a message, where we will include our weather summary message, the phone number we have on twilio and our own phone number. 

In [25]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                    body=summary,
                    from_=PHONE_NUMBER,
                    to=MY_NUMBER
                )

print('Message Sent ' + message.sid)

Message Sent SM46f9c72a0eb28153df65f44537d71ce0


# Result 

* The message is sent from our twilio phone number to our cell phone

<img src="WhatsApp Image 2023-07-02 at 16.44.36.jpeg" />